## Path Setup

In [24]:
import os
import pandas as pd

RAW_PATH = r"C:\Users\adana\etl-online-retail\online_retail.csv"
OUT_DIR = r"C:\Users\adana\etl-online-retail\processed"

os.makedirs(OUT_DIR, exist_ok=True)

### Read raw data

In [35]:
df = pd.read_csv(RAW_PATH, encoding_errors="ignore")
print("Initial shape:", df.shape)

Initial shape: (541909, 8)


### Standardize column names

In [40]:
df.columns = df.columns.str.strip().str.lower()
df.columns

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

### Drop duplicates

In [47]:
df = df.drop_duplicates()
print("Shape after removal of duplicate rows:", df.shape)

Shape after eemoval of duplicate rows: (536641, 8)


### Remove rows with missing critical values

In [50]:
df = df.dropna(subset=["invoiceno", "stockcode", "quantity", "invoicedate", "unitprice", "customerid"])
print("Shape after removal of rows having missing critical values:", df.shape)

Shape after removal of rows having missing critical values: (401604, 8)


### Standardize column data types (date, quantity, price)

In [53]:
df["invoicedate"] = pd.to_datetime(df["invoicedate"], errors="coerce")
df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce").astype(int)
df["unitprice"] = pd.to_numeric(df["unitprice"], errors="coerce")

### Remove invalid transactions (returns, cancellations)

In [58]:
df = df[df["quantity"] > 0]
df = df[df["unitprice"] > 0]

### Create calculated fields (sales amount and date parts)

In [61]:
df["total_amount"] = df["quantity"] * df["unitprice"]
df["year"] = df["invoicedate"].dt.year
df["month"] = df["invoicedate"].dt.month
df["day"] = df["invoicedate"].dt.day

### Build Dimension Tables (Products, Customers, Dates)

In [64]:
dim_products = df[["stockcode", "description", "unitprice"]].drop_duplicates().reset_index(drop=True)
dim_customers = df[["customerid", "country"]].drop_duplicates().reset_index(drop=True)
dim_date = df[["invoicedate"]].drop_duplicates().reset_index(drop=True)
dim_date["date_id"] = dim_date["invoicedate"].dt.strftime("%Y%m%d").astype(int)
dim_date["year"] = dim_date["invoicedate"].dt.year
dim_date["month"] = dim_date["invoicedate"].dt.month
dim_date["day"] = dim_date["invoicedate"].dt.day

### Build Fact Table (Sales Transactions)

In [67]:
df["date_id"] = df["invoicedate"].dt.strftime("%Y%m%d").astype(int)
fact_sales = df[["invoiceno", "customerid", "stockcode", "date_id", "quantity", "unitprice", "total_amount"]]

### Export Fact and Dimension Tables to CSV

In [72]:
fact_sales.to_csv(os.path.join(OUT_DIR, "fact_sales.csv"), index=False)
dim_products.to_csv(os.path.join(OUT_DIR, "dim_products.csv"), index=False)
dim_customers.to_csv(os.path.join(OUT_DIR, "dim_customers.csv"), index=False)
dim_date.to_csv(os.path.join(OUT_DIR, "dim_date.csv"), index=False)

### ETL Completion and Summary

In [75]:
print("ETL Completed ✅")
print("Fact rows:", len(fact_sales))

ETL Completed ✅
Fact rows: 392692
